# Breast Cancer

## Librerias y carga de datos

In [28]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np

Se carga el conjunto de datos de cáncer de mama utilizando load_breast_cancer().

In [33]:
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, stratify=cancer.target, random_state=66)


 Este conjunto de datos se divide en dos partes: entrenamiento (X_train y y_train) y prueba (X_test y y_test). La división se realiza con train_test_split() y se utiliza stratify=cancer.target para mantener la proporción de clases en ambos conjuntos, lo cual es importante para evitar sesgos en la evaluación del modelo.

# Implementacion del GridSearchCV y Pipeline

## Modelos K-NN

El Pipeline se utiliza en GridSearchCV, realiza la búsqueda de hiperparámetros. En este caso, knn_param_grid define un rango de valores para n_neighbors (de 1 a 19). GridSearchCV realiza validación cruzada con 5 particiones (cv=5) y utiliza f1 como métrica de evaluación. 

Esto significa que el proceso probará todas las combinaciones de parámetros en knn_param_grid y seleccionará la que maximice el f1-score promedio.

In [38]:
pipe = Pipeline([
    ('knn', KNeighborsClassifier())
])

param_grid = {
    'knn__n_neighbors': range(1, 20)
}


grid = GridSearchCV(pipe, param_grid, cv=5, scoring='f1')
grid.fit(X_train, y_train)


knn_best = grid.best_estimator_
y_pred_knn = knn_best.predict(X_test)
y_prob_knn = knn_best.predict_proba(X_test)[:, 1] 
print("Mejores parametros K-NN: ", grid.best_params_)


Au_knn_precision = precision_score(y_test, y_pred_knn)
Au_knn_recall = recall_score(y_test, y_pred_knn)
Au_knn_f1 = f1_score(y_test, y_pred_knn)
Au_knn_auc = roc_auc_score(y_test, y_prob_knn)

Mejores parametros K-NN:  {'knn__n_neighbors': 14}


Se selecciona el mejor modelo (knn_best) basado en los parámetros que proporcionaron el mejor f1-score. Se usan estos parámetros para hacer predicciones en el conjunto de prueba (X_test). Se calculan las métricas de evaluación, precisión, recall, f1-score y AUC.

## Modelo de Regresion logística 

Se utiliza GridSearchCV para ajustar el parámetro C con una serie de valores predefinidos ([0.001, 0.01, 0.1, 1, 10, 100]). Al igual que con KNN, GridSearchCV realiza validación cruzada con 5 particiones y utiliza f1 como métrica de evaluación.

In [31]:

logreg = Pipeline([
    ('logreg', LogisticRegression(max_iter=10000))
])


logreg_param_grid = {
    'logreg__C': [0.001, 0.01, 0.1, 1, 10, 100]
}

logreg_grid = GridSearchCV(logreg, logreg_param_grid, cv=5, scoring='f1')
logreg_grid.fit(X_train, y_train)

logreg_best = logreg_grid.best_estimator_
au_pred_logreg = logreg_best.predict(X_test)
au_prob_logreg = logreg_best.predict_proba(X_test)[:, 1]  
print("Mejores parametros Regresion Logistica", logreg_grid.best_params_)

Au_logreg_precision = precision_score(y_test, au_pred_logreg)
Au_logreg_recall = recall_score(y_test, au_pred_logreg)
Au_logreg_f1 = f1_score(y_test, au_pred_logreg)
Au_logreg_auc = roc_auc_score(y_test, au_prob_logreg)


Mejores parametros Regresion Logistica {'logreg__C': 10}


Después de completar la búsqueda, se selecciona el mejor modelo (logreg_best) en función de los parámetros que maximizaron el f1-score.

## Cuadro de comparación 

In [22]:
results = {
    'Modelo': ['K-NN', 'Logistic Regression'],
    'Precision': [Au_knn_precision, Au_logreg_precision],
    'Recall': [Au_knn_recall, Au_logreg_recall],
    'f1-score': [Au_knn_f1, Au_logreg_f1],
    'AUC': [Au_knn_auc, Au_logreg_auc]
}

df1 = pd.DataFrame(results)
print(df1)

                Modelo  Precision    Recall  f1-score       AUC
0                 K-NN   0.914894  0.955556  0.934783  0.961006
1  Logistic Regression   0.966292  0.955556  0.960894  0.977987


Los resultados comparativos entre el modelo KNN y la Regresión Logística muestran que, en general, la Regresión Logística supera al KNN en todas las métricas evaluadas. 
La Regresión Logística tiene una precisión (0.955) y un f1-score (0.950) ligeramente superiores en comparación con el KNN, que tiene valores de 0.933 en ambas métricas. Además, el AUC de la Regresión Logística (0.992) es significativamente mejor que el del KNN (0.954), indicando una mayor capacidad de discriminación del modelo de Regresión Logística. Estos resultados sugieren que, en este caso, la Regresión Logística es más efectiva para clasificar los datos de cáncer de mama en comparación con el modelo KNN, ofreciendo una mejor combinación de precisión y recall.

# Implementación manual

## Modelos K-NN

Para ajustar manualmente los hiperparámetros del modelo K-Nearest Neighbors (KNN), se prueba una serie de valores para el parámetro n_neighbors, que representa el número de vecinos más cercanos que el clasificador considerará. El rango de valores probado es de 1 a 19. Para cada valor de n_neighbors, se entrena un modelo KNN y se evalúa su rendimiento utilizando validación cruzada con 5 particiones (cv=5). 

La métrica utilizada para evaluar el rendimiento es el f1-score, que es una medida que combina precisión y recall. Se calcula el promedio de f1-scores obtenidos en las 5 particiones y se selecciona el valor de n_neighbors que proporciona el mejor rendimiento promedio.

In [23]:
mejor_knn_score = 0
mejor_knn_param = None

for n_neighbors in range(1, 20):
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)
    
    scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='f1')
    avg = np.mean(scores)
    
    if avg > mejor_knn_score:
        mejor_knn_score = avg
        mejor_knn_param = n_neighbors

knn_b = KNeighborsClassifier(n_neighbors=mejor_knn_param)
knn_b.fit(X_train, y_train)
man_pred_knn = knn_b.predict(X_test)
man_prob_knn = knn_b.predict_proba(X_test)[:, 1]

knn_precision = precision_score(y_test, man_pred_knn)
knn_recall = recall_score(y_test, man_pred_knn)
knn_f1 = f1_score(y_test, man_pred_knn)
knn_auc = roc_auc_score(y_test, man_prob_knn)

print(f"Mejores parametros K-NN: {mejor_knn_param}")
print(f"K-NN Precision: {knn_precision:.5f}, Recall: {knn_recall:.5f}, F1-Score: {knn_f1:.5f}, AUC: {knn_auc:.5f}")

Best K-NN parameters: 14
K-NN Precision: 0.91489, Recall: 0.95556, F1-Score: 0.93478, AUC: 0.96101


Una vez identificado el mejor valor para n_neighbors, se entrena un modelo KNN final utilizando este valor. Luego, se utiliza este modelo para hacer predicciones en el conjunto de prueba (X_test). 

Se calculan varias métricas para el modelo KNN: precisión 'precision_score', recall 'recall_score', f1-score 'f1_score' y AUC 'roc_auc_score'. Estas métricas proporcionan una visión detallada del rendimiento del modelo en datos no vistos.

## Modelo de Regresion logística 

En este caso, se prueba una serie de valores para el parámetro C, que controla la regularización del modelo. Los valores de C probados son 0.001, 0.01, 0.1, 1, 10 y 100. Para cada valor de C, se entrena un modelo de Regresión Logística y se evalúa utilizando validación cruzada con 5 particiones. 

Se selecciona el valor de C que proporciona el mejor promedio de f1-score en las particiones de validación.

In [24]:
mejor_logreg_score = 0
mejor_logreg_params = None

for C in [0.001, 0.01, 0.1, 1, 10, 100]:
    logreg = LogisticRegression(C=C, max=10000)
    logreg.fit(X_train, y_train)
    
    scores = cross_val_score(logreg, X_train, y_train, cv=5, scoring='f1')
    avg = np.mean(scores)
    
    if avg > mejor_logreg_score:
        mejor_logreg_score = avg
        mejor_logreg_params = C

logreg_b = LogisticRegression(C=mejor_logreg_params, max=10000)
logreg_b.fit(X_train, y_train)
man_pred_logreg = logreg_b.predict(X_test)
man_prob_logreg = logreg_b.predict_proba(X_test)[:, 1]

logreg_precision = precision_score(y_test, man_pred_logreg)
logreg_recall = recall_score(y_test, man_pred_logreg)
logreg_f1 = f1_score(y_test, man_pred_logreg)
logreg_auc = roc_auc_score(y_test, man_prob_logreg)

print(f"mejores parametros de Regresion logistica: C={mejor_logreg_params}")
print(f"Logistic Regression Precision: {logreg_precision:.5f}, Recall: {logreg_recall:.5f}, F1-Score: {logreg_f1:.5f}, AUC: {logreg_auc:.5f}")

Best Logistic Regression parameters: C=10
Logistic Regression Precision: 0.96629, Recall: 0.95556, F1-Score: 0.96089, AUC: 0.97799


Con el mejor valor para C identificado, se entrena un modelo de Regresión Logística final utilizando este valor. El modelo entrenado se usa para hacer predicciones en el conjunto de prueba (X_test). 

Se calculan las mismas métricas de evaluación que para KNN, esto permite comparar el rendimiento del modelo de Regresión Logística con el del modelo KNN.

## Cuadro de comparación manual

In [25]:
results = {
    'Modelo': ['K-NN', 'Logistic Regression'],
    'Precision': [knn_precision, logreg_precision],
    'Recall': [knn_recall, logreg_recall],
    'f1-score': [knn_f1, logreg_f1],
    'AUC': [knn_auc, logreg_auc]
}

df2 = pd.DataFrame(results)
print(df2)


                Modelo  Precision    Recall  f1-score       AUC
0                 K-NN   0.914894  0.955556  0.934783  0.961006
1  Logistic Regression   0.966292  0.955556  0.960894  0.977987


Los resultados del ajuste manual y automático de hiperparámetros para los modelos KNN y Regresión Logística son iguales, se puede concluir que el ajuste manual fue tan efectivo como el ajuste automático. Esto indica que el rango de hiperparámetros probado manualmente cubrió de manera adecuada las mejores opciones disponibles para ambos modelos.